# Going through the [MSFT documentation on the Python Library for Digital Twin](https://docs.microsoft.com/en-us/python/api/overview/azure/digitaltwins-core-readme-pre?view=azure-python&source=docs)

That doc is aweful. So I went through the code and tested everythign out. As well as added some things to make it easier. I'll create some more usefull notebooks. I just created this to work through the problems in the documentation. 

Unlike the doc, the code examples in this notebook will run. Plus I've take out a lot of convoluted steps that make no sense to someone who's just getting into Digital Twin. Note that I'm using the `master` branch from github and not the pip installed sdk. 


In [10]:
from azure.identity import AzureCliCredential
from azure.digitaltwins.core import DigitalTwinsClient

The ADT docs use `DefaultAzureCredential` as an example. That takes a lot of extra time to set up, especially for exploring ADT locally. For REPL and local work, use the AzureCliCredential instead. It has way lest setup. 

In [11]:
from azure.identity import AzureCliCredential
azure_cli = AzureCliCredential()

In [12]:
serviceClient = DigitalTwinsClient(
    "home-test-twin.api.wcus.digitaltwins.azure.net", azure_cli)


In [14]:
for i in serviceClient.list_models():
    print(i)

{'additional_properties': {}, 'display_name': {'en': 'SampleModel'}, 'description': {}, 'id': 'dtmi:com:contoso:SampleModel;1', 'upload_time': datetime.datetime(2020, 9, 2, 23, 41, 42, 168566, tzinfo=<FixedOffset '+00:00'>), 'decommissioned': False, 'model': None}
{'additional_properties': {}, 'display_name': {'en': 'Base Consumer'}, 'description': {}, 'id': 'dtmi:example:grid:consumer:baseConsumer;1', 'upload_time': datetime.datetime(2020, 11, 2, 13, 45, 38, 349773, tzinfo=<FixedOffset '+00:00'>), 'decommissioned': False, 'model': None}
{'additional_properties': {}, 'display_name': {'en': 'SampleModel'}, 'description': {}, 'id': 'dtmi:example:SampleModel;1', 'upload_time': datetime.datetime(2020, 11, 4, 4, 44, 55, 817055, tzinfo=<FixedOffset '+00:00'>), 'decommissioned': False, 'model': None}
{'additional_properties': {}, 'display_name': {'en': 'Patron'}, 'description': {'en': 'As an example, contains all of the properties possible in teh DTDL.'}, 'id': 'dtmi:billmanh:patron;1', 'uplo

In [52]:
i.id

'dtmi:billmanh:patron;1'

In [53]:
# Get a model
get_model = serviceClient.get_model(i.id)
print('Get Model:')
get_model.as_dict()

Get Model:


{'display_name': {'en': 'Patron'},
 'description': {'en': 'As an example, contains all of the properties possible in teh DTDL.'},
 'id': 'dtmi:billmanh:patron;1',
 'upload_time': '2020-11-09T01:29:21.051879Z',
 'decommissioned': False}

In [54]:
i.additional_properties

{}

In [28]:
i.description

{'en': 'As an example, contains all of the properties possible in teh DTDL.'}

In [24]:
import uuid
digital_twin_id = 'digitalTwin-' + str(uuid.uuid4())
digital_twin_id

'digitalTwin-3fefef71-2482-465a-9f68-09224183a4a3'

In [37]:
twin_data = {
  "$metadata": {
    "$model": i.id
  }
}

created_twin = serviceClient.upsert_digital_twin(digital_twin_id, twin_data)
created_twin

{'$dtId': 'digitalTwin-3fefef71-2482-465a-9f68-09224183a4a3',
 '$etag': 'W/"f3cd72a7-91ee-4349-a2c0-5f4ce14d572e"',
 '$metadata': {'$model': 'dtmi:billmanh:patron;1'}}

Note that the resonse is just a dict, not a twin object or something. 

In [41]:
get_twin = serviceClient.get_digital_twin(digital_twin_id)
get_twin

{'$dtId': 'digitalTwin-3fefef71-2482-465a-9f68-09224183a4a3',
 '$etag': 'W/"f3cd72a7-91ee-4349-a2c0-5f4ce14d572e"',
 '$metadata': {'$model': 'dtmi:billmanh:patron;1'}}

In [51]:
query_expression = 'SELECT * FROM digitaltwins'
query_result = serviceClient.query_twins(query_expression)
print('DigitalTwins:')
for twin in query_result:
    print("    -: {}".format(twin["$dtId"]))

DigitalTwins:
    -: sampleTwin-1
    -: sampleTwin-0
    -: sampleTwin-2
    -: digitalTwin-3fefef71-2482-465a-9f68-09224183a4a3


This is where the doc breaks down again. There are some abmiguities between _components_ and _contents_ mentioned in the c# tutorial. 

In [50]:
component_path = "contents"
get_component = serviceClient.get_component(digital_twin_id, component_path)
print('Get Component:')
print(get_component)

ResourceNotFoundError: (ComponentNotFound) digitalTwin-3fefef71-2482-465a-9f68-09224183a4a3 does not have component contents Please verify that the component path is correct. You can get components by getting the twin, See example http://aka.ms/TwinGetSwSmpl.

In [45]:
options = {
    "patchDocument": {
    "ComponentProp1": "value2"
    }
}
serviceClient.update_component(digital_twin_id, component_path, options)

HttpResponseError: Operation returned an invalid status 'Bad Request'